In [ ]:
# Importing finviz API
from finvizfinance.quote import finvizfinance
# Importing neo4j drivers
# Graphdatabase to run driver
# Routing control to decide read/write operations
from neo4j import GraphDatabase, RoutingControl
#Importing warnings to ignore and de-clutter the output
import warnings
warnings.filterwarnings('ignore')

# Initialising URI, Authorisation and driver to connect to neo4j session through python
URI = "neo4j://127.0.0.1:7687"
AUTH = ("neo4j", "V3rYstr0ngp@sSw0rd")
driver = GraphDatabase.driver(auth=AUTH, uri=URI, notifications_min_severity="OFF")

In [ ]:
# Getting stock of Tesla into stock data item
stock = finvizfinance('tsla')
# Getting stock data into tesla_fundament
tesla_fundament = stock.ticker_fundament()
tesla_fundament

In [ ]:
# Alternate method to get stock data
tesla_fundament = finvizfinance('tsla').ticker_fundament()
tesla_fundament

In [ ]:
# Experimental code written to get company names
# f = open("Stock tickers.txt", "r")

# names = []

# for x in f:
#     stock_data = finvizfinance(x[:-1]).ticker_fundament()
#     names.append(stock_data["Company"])

In [ ]:
# Experimnetal code written to add tickers due to a previous oversight
# f = open("500 tickers.txt", "r")

# i = 0

# for x in f:
#     string = str(x[:-1])
#     addTicker(names[i], string)
#     i += 1
    
# f.close()

In [ ]:
# Function to add ticker to node given company name (no longer needed)
# def addTicker(company_name, ticker):
#     with GraphDatabase.driver(auth=AUTH, uri=URI) as driver:
#         driver.execute_query(
#         "MATCH (n:stock {Company : $company})"
#         "SET n.Ticker = $ticker",
#         company = company_name,
#         ticker = ticker,
#         database_="fin-proj",
#         routing_=RoutingControl.WRITE
#     )

In [ ]:
def addDatabase(database):
    # Test query to return databases
    result = driver.execute_query(
            "show databases yield name",
            database_="neo4j", routing_=RoutingControl.READ
            )
        
    # Test code to check if specified database exists in database list
    flag = False
    for record in result.records:
        if record["name"] == database:
            flag = True
        
    if(flag):
        print(f"Databse {database} already exists")
    else:
        # Query to create database
        driver.execute_query(
            "create database $database",
            database=database,
            database_="neo4j", routing_=RoutingControl.WRITE
        )
        print("Added database:", database)

In [ ]:
def addStock(stock_fundament: dict, database: str):
    # Formatting stock_fundament to input as query into neo4j
    properties = ", ".join([f"`{key}` : $`{key}`" for key in stock_fundament.keys()])
    # Creating query data item as string to pass into driver query
    query = f"CREATE (:stock {{{properties}}})"

    # Adding stock data to database
    driver.execute_query(query, stock_fundament, database_=database, routing_=RoutingControl.WRITE)
    

In [ ]:
# Adds the ticker to stock fundament since ticker is not 
# in the fundament by default
def fundament_modifier(stock_ticker: str) -> dict:
    stock_fundament = finvizfinance(stock_ticker).ticker_fundament()
    stock_fundament["Ticker"] = stock_ticker
    return stock_fundament

In [ ]:
# Building the database using given file
# File contains tickers of all stocks to be added to the database
def buildDatabase(ticker_file: str, database: str):
    f = open(ticker_file, "r")
    
    for x in f:
        # Getting ticker
        ticker = str(x[:-1])
        
        # Checking if stock already in the database
        res, _, __ = driver.execute_query(
            "MATCH (n:stock {Ticker: $Ticker}) "
            "RETURN n ",
            Ticker=ticker,
            database_=database,
            routing_=RoutingControl.READ
        )
        
        # If stock in databse, we do not add it
        if res:
            print(f"Stock {ticker} already exists")
            continue
        
        # Adding the stock
        addStock(fundament_modifier(str(x[:-1])), database)
        print("Added stock:", ticker)

In [11]:
def getData(Company: str):
    #Getting stock data into new_res to use as data
    new_res, _, _ = driver.execute_query(
        "match (n: stock {Company : $Company})"
        "return n",
        Company = Company,
        database_="experimental",
        routing_=RoutingControl.READ
    )

In [12]:
# Older version of createSectors() function for linking stocks of same sector
# New version has been split into creating sectors, then linking stocks to the sector
# def createSectors():
#     #Creating a relationship between all stocks that have the same sector
#     with GraphDatabase.driver(auth=AUTH, uri=URI) as driver:
#         driver.execute_query(
#             "MATCH (u1:stock), (u2:stock) "
#             "WHERE u1.Sector = u2.Sector AND elementId(u1) < elementId(u2) "
#             "CREATE (u1)-[:same_sector]->(u2) ",
#             database_="experimental",
#             routing_=RoutingControl.WRITE        
#         )

In [ ]:
# Function to create sector nodes in the database based
# on all distinct sectors of available stock information
def createSectors(database: str):
    # Getting all distinct sectors
    result, _, __ = driver.execute_query(
        "MATCH (n:stock) "
        "RETURN DISTINCT n.Sector as Sectors ",
        database_=database,
        routing_=RoutingControl.READ
    )

    for x in result:
        sector = x["Sectors"]
        
        # Checking if current sector is already in our database
        res, _, __ = driver.execute_query(
            "MATCH (n:sector {name:$name}) "
            "RETURN n as name ",
            name=sector,
            database_=database,
            routing_=RoutingControl.READ
        )
        
        if res:
            print(f"Sector {sector} is already in the database")
            continue
        
        # Adding the sector node to the database
        driver.execute_query(
            "CREATE (:sector {name:$name}) ",
            name=sector,
            database_=database,
            routing_=RoutingControl.WRITE
        )
        print("Added sector:", sector)

In [ ]:
# Creating relationships from stocks in a particular sector
# to corresponding sector node if the relationship does not exist already
def joinSector(sector: str, database: str):
    driver.execute_query(
        "MATCH (sec:sector) "
        "WHERE sec.name = $sector "
        "MATCH (stk:stock) "
        "WHERE stk.Sector = $sector "
        "MERGE (stk)-[:in_sector]->(sec) ",
        sector=sector,
        database_=database,
        routing_=RoutingControl.WRITE
    )
    print("Joined all stocks to sector:", sector)

In [ ]:
# Function to find all distinct sectors of stocks in the database
# and join all stocks of one sector to that sector
def joinAllSectors(database: str):
    # Getting all distinct sectors
    names, _, __ = driver.execute_query(
        "MATCH (s:sector) "
        "RETURN s.name as Names ",
        database_=database,
        routing_=RoutingControl.READ
    )
    
    for temp in names:
        sector = temp["Names"]
        # Joining all stocks of each sector
        joinSector(sector, database)

In [19]:
if __name__ == "__main__":
    # Function to build the database using S&P 500 tickers at 2:30 AM IST on 28-09-25
    # buildDatabase("500 tickers.txt", "experimental")
    # createSectors("experimental")
    joinAllSectors("experimental")

Joined all stocks to sector: Energy
Joined all stocks to sector: Financial
Joined all stocks to sector: Utilities
Joined all stocks to sector: Real Estate
Joined all stocks to sector: Industrials
Joined all stocks to sector: Healthcare
Joined all stocks to sector: Consumer Cyclical
Joined all stocks to sector: Basic Materials
Joined all stocks to sector: Technology
Joined all stocks to sector: Communication Services
Joined all stocks to sector: Consumer Defensive
